# Performance Testing with Computing the Mandlebrot Set

This sample was executed on a DSVM on a Standard_D2_v2 in Azure. 

This code below also uses a few other cluster config files titled: 
- "10_core_cluster.json" 
- "20_core_cluster.json"
- "40_core_cluster.json"
- "80_core_cluster.json"

Each of the cluster config files above are used by the doAzureParallel package. They all define static clusters (minNodes = maxNodes) and use the Standard_F2 VM size. 

Install package dependencies for doAzureParallel

In [ ]:
install.packages(c('httr','rjson','RCurl','digest','foreach','iterators','devtools','curl','jsonlite','mime'))

Install doAzureParallel and rAzureBatch from github

In [ ]:
library(devtools)
install_github("Azure/rAzureBatch")
install_github("Azure/doAzureParallel")

Install *microbenchmark* package and other utilities

In [ ]:
install.packages("microbenchmark")
library(microbenchmark)
library(reshape2)
library(ggplot2)

Define function to compute the mandlebrot set.

In [ ]:
vmandelbrot <- function(xvec, y0, lim)
{
  mandelbrot <- function(x0,y0,lim)
  {
    x <- x0; y <- y0
    iter <- 0
    while (x^2 + y^2 < 4 && iter < lim)
    {
      xtemp <- x^2 - y^2 + x0
      y <- 2 * x * y + y0
      x <- xtemp
      iter <- iter + 1
    }
    iter
  }
 
  unlist(lapply(xvec, mandelbrot, y0=y0, lim=lim))
}

The local execution is performed on a single Standard_D2_V2 DSVM in Azure. We use the doParallel package and use both cores for this performance test

In [ ]:
localExecution <- function() {
  print("doParallel")
  library(doParallel)
  cl<-makeCluster(2)
  registerDoParallel(cl)
 
  x.in <- seq(-2, 1.5, length.out=1080)
  y.in <- seq(-1.5, 1.5, length.out=1080)
  m <- 1000
  mset <- foreach(i=y.in, .combine=rbind, .export = "vmandelbrot") %dopar% vmandelbrot(x.in, i, m)
}

The Azure Execution takes in a pool_config JSON file and will use doAzureParallel.

In [ ]:
azureExecution <- function(pool_config) {
  print("doAzureParallel")
  library(doAzureParallel)
  pool <- doAzureParallel::makeCluster(pool_config)
  registerDoAzureParallel(pool)
 
  x.in <- seq(-2, 1.5, length.out=1080)
  y.in <- seq(-1.5, 1.5, length.out=1080)
  m <- 1000
  mset <- foreach(i=y.in, .combine=rbind, .options.azure = list(chunkSize=10), .export = "vmandelbrot") %dopar% vmandelbrot(x.in, i, m)
}

Using the *microbenchmark* package, we test the difference in performance when running the same code to calculate the mandlebrot set on a single machine (localExecution), a cluster of 10 cores, a cluster of 20 cores, and finally a cluster of 40 cores.

In [ ]:
op <- microbenchmark(
  doParLocal=localExecution(),
  doParAzure_10cores=azureExecution("10_core_cluster.json"),
  doParAzure_20cores=azureExecution("20_core_cluster.json"),
  doParAzure_40cores=azureExecution("40_core_cluster.json"),
  times=5L)

In [ ]:
print(op)

In [ ]:
plot(op)